In [1]:
import cv2
import numpy as np
import pandas as pd
import imutils
bg = None
import pickle
import cv2
import os

In [2]:
def run_avg(image , aWeight):
    global bg
    if bg is None:
        bg = image.copy().astype('float')
        return 
    cv2.accumulateWeighted(image, bg , aWeight)

In [3]:
def segment(image, threshold = 15):
    diff = cv2.absdiff(bg.astype('uint8') , image)
    thresholded = cv2.threshold(diff,threshold, 255 , cv2.THRESH_BINARY)[1]
    (cnts, _) = cv2.findContours(thresholded.copy() , cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_SIMPLE)
    if len(cnts) == 0:
        return
    else:
        segmented = max(cnts, key = cv2.contourArea)
        return (thresholded , segmented)

In [4]:
loc = r'D://leapgestrecog//leapGestRecog//leapGestRecog//0'

In [5]:
lookup = dict()
reverselookup = dict()
count = 0
for j in os.listdir(loc + str(0)):
    if not j.startswith('.'):                              
        lookup[j] = count
        reverselookup[count] = j
        count = count + 1
lookup

{'01_palm': 0,
 '02_l': 1,
 '03_fist': 2,
 '04_fist_moved': 3,
 '05_thumb': 4,
 '06_index': 5,
 '07_ok': 6,
 '08_palm_moved': 7,
 '09_c': 8,
 '10_down': 9}

In [7]:
if __name__ == "__main__":    
    aWeight = 0.5    
    camera = cv2.VideoCapture(0)    
    filename = '_model_.sav'
    model = pickle.load(open(filename, 'rb'))
    top, right, bottom, left = 10, 350, 225, 590    
    num_frames = 0  
    predict_img = None
    result = None
    while(True):       
        (grabbed, frame) = camera.read()        
        frame = cv2.flip(frame, 1)        
        
        clone = frame.copy()       
        (height, width) = frame.shape[:2]        
        roi = frame[top:bottom, right:left]        
       
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)        
        if num_frames < 30:
            run_avg(gray, aWeight)
        else:           
            hand = segment(gray)            
            if hand is not None:                
                (thresholded, segmented) = hand                
                cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255) , 3)
                cv2.imshow("Thresholded", thresholded)
                predict_img = thresholded                
                predict_img = cv2.resize(predict_img , (120,320))
                predict_img = np.array(predict_img, dtype = 'float32')
                y_pred = model.predict(predict_img.reshape(1,120,320,1))
                result = np.argmax(y_pred)
                
                result = reverselookup[result]
                
                 
                
        font = cv2.FONT_HERSHEY_SIMPLEX        
        color = (255, 255, 255)
        i = 0
        cv2.putText(clone,  
            result,  
            (350, 250),  
            font, 1,  
            color,  
            2,  
            cv2.LINE_AA)
            
            
        cv2.rectangle(clone, (left, top), (right, bottom), (0,255,0), 2)        
        num_frames += 1   
         
        
        #print(result)
        cv2.imshow("Video Feed", clone)        
        keypress = cv2.waitKey(1) & 0xFF        
        if keypress == ord("q"):
            break
camera.release()
cv2.destroyAllWindows()